In [ ]:
pip install pandas

In [ ]:
import pandas as pd

In [ ]:
pip install openpyxl

In [ ]:
data = pd.read_excel("data.xlsx")
df = pd.DataFrame(data)

final_data = []

for row in df.itertuples(index=True):
    dat = {
        "id": row.Index,
        "name": row._1,
        "email": row._2
    }
    final_data.append(dat)

In [ ]:
new_data = final_data.copy()
new_data[10]["name"] = "Babita Adhikari"
import json

# Assuming new_data is already updated
with open("updated_data.json", "w") as f:
    json.dump(new_data, f, indent=4)

In [33]:
import json
import os

# Load the original data
with open("updated_data.json") as f:
    data = json.load(f)

# Add certificate path for each item
for item in data:
    cert_id = item["id"]  # assuming id is 0–35
    image_filename = f"certificates/untitled{cert_id}.jpg"

    # Optional: check if file exists
    if os.path.exists(image_filename):
        item["certificate"] = image_filename
    else:
        item["certificate"] = None  # or handle missing files

# Save the updated data
with open("data_with_certificates.json", "w") as f:
    json.dump(data, f, indent=4)


In [ ]:
import json
import smtplib
import ssl
from email.message import EmailMessage
import os

# Your email credentials
SENDER_EMAIL = ""
SENDER_PASSWORD = ""

# Load participant data
with open("data_with_certificates.json") as f:
    participants = json.load(f)

# SMTP settings
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# Email content
SUBJECT = "Your Participation Certificate"
BODY_TEMPLATE = """\
Dear {name},

Thank you for participating in our event. Please find your certificate attached.

Best regards,
The Organizing Team
"""

# Send emails
context = ssl.create_default_context()

with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
    server.starttls(context=context)
    server.login(SENDER_EMAIL, SENDER_PASSWORD)

    for person in participants[0:5]:
        name = person["name"]
        email = person["email"]
        cert_path = person["certificate"]

        if not os.path.exists(cert_path):
            print(f"Certificate not found for {name}, skipping.")
            continue

        msg = EmailMessage()
        msg["From"] = SENDER_EMAIL
        msg["To"] = "bibekjoshi34@gmail.com"  # ← use the actual recipient
        msg["Subject"] = SUBJECT
        msg.set_content(BODY_TEMPLATE.format(name=name))

        with open(cert_path, "rb") as cert_file:
            cert_data = cert_file.read()
            cert_name = os.path.basename(cert_path)
            msg.add_attachment(cert_data, maintype="image", subtype="jpeg", filename=cert_name)

        try:
            server.send_message(msg)
            print(f"✅ Email sent to {name} ({email})")
        except Exception as e:
            print(f"❌ Failed to send email to {name} ({email}): {e}")


SMTPAuthenticationError: (534, b'5.7.9 Please log in with your web browser and then try again. For more\n5.7.9 information, go to\n5.7.9  https://support.google.com/mail/?p=WebLoginRequired 98e67ed59e1d1-30f36513276sm1798113a91.44 - gsmtp')